### Arch Linux Packages (non-AUR Arch Linux packages)

In addition to the AUR, we also have core Arch Linux packages that are not included in the AUR. The script below shows how we can scrape data on these packages. I later realized that these packages are available in the [sitemap](https://www.archlinux.org/sitemap-packages.xml). 

In [1]:
from selenium import webdriver
import re
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
import requests

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import networkx

In [ ]:
os.chdir('../html/package-pages/')
driver = webdriver.PhantomJS()
base_url = 'https://www.archlinux.org/packages/?page='
for i in range(1,101):
    search_url = base_url + str(i) + "&"
    print(search_url)
    driver.get(search_url)
    time.sleep(2 + np.random.random())
    html = driver.page_source.encode('utf-8')
    name = "package_page_" + str(i)
    package_list = open(name+'.txt', 'w+')
    package_list.write(str(html))
    package_list.close()

In [ ]:
os.chdir('../html/package-pages/')
files = os.listdir()
dict_list = []
for file in files:     
    f = open(file, 'r')
    html = f.read()
    b = BeautifulSoup(html, 'lxml')
    try: 
        packages = b.find_all('tr')[1:]
        for package in packages:
            data = package.find_all('td')
            data_dict = {
                         "arch": data[0].text,
                         "repo": data[1].text,
                         "name": data[2].text,
                         "link": data[2].find('a')['href'],
                         "version": data[3].text,
                         "description": data[4].text, 
                         "last_updated": data[5].text,
                         "flag_date": data[6].text
                        }
            dict_list.append(data_dict)
    except:
        print(file)
    b.decompose()
    f.close()

In [ ]:
cols = ['arch', 'repo', 'name', 'link', 'version', 'description', 'last_updated', 'flag_date']
df = pd.DataFrame(dict_list, columns=cols)
df = df.drop_duplicates()
df.to_csv('../csv/arch_pack_data.csv')

### Core Arch Package Details

In [2]:
df = pd.read_csv('../csv/arch_pack_data.csv', index_col=0)